In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
# --- 1. Load and Prepare Your Labeled Data ---
print("Loading financial sentiment data...")
try:
    # The 'latin1' encoding is often needed for this specific dataset
    df = pd.read_csv('./dev/data/sentiment_data.csv', encoding='latin1')
except FileNotFoundError:
    print("Error: 'sentiment_data.csv' not found. Make sure it's in the same folder.")

# Drop any rows with missing data
df.dropna(inplace=True)

# Create a mapping from category name to integer ID
unique_labels = df['Sentiment'].unique()
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for i, label in enumerate(unique_labels)}

# Add a new column with the integer labels
df['label'] = df['Sentiment'].map(label2id)

print(f"Loaded {len(df)} labeled sentences.")
print(f"Categories found: {list(unique_labels)}")

Loading financial sentiment data...
Loaded 5842 labeled sentences.
Categories found: ['positive', 'negative', 'neutral']


In [4]:
# --- 2. Load Pre-trained FinBERT Tokenizer and Model ---
model_name = "ProsusAI/finbert"
print(f"Loading tokenizer and model for '{model_name}'...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)

Loading tokenizer and model for 'ProsusAI/finbert'...


In [5]:
# --- 3. Create a PyTorch Dataset ---
# Split your data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Tokenize the text
train_encodings = tokenizer(train_df['Sentence'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['Sentence'].tolist(), truncation=True, padding=True)

# Create a custom PyTorch Dataset class
class FinancialDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
        
train_dataset = FinancialDataset(train_encodings, train_df['label'].tolist())
val_dataset = FinancialDataset(val_encodings, val_df['label'].tolist())

In [ ]:
# --- 4. Fine-Tune the Model with the Trainer API ---
print("Configuring training...")

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./dev/checkpoints',          # Directory to save results
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=16,   # Batch size for training
    per_device_eval_batch_size=16,    # Batch size for evaluation
    logging_dir='./dev/logs',            # Directory for storing logs
    logging_steps=10,
    eval_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save checkpoint at the end of each epoch
    load_best_model_at_end=True,     # Load the best model found during training
)

# Define a function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Start training!
print("Starting fine-tuning...")
trainer.train()
print("Fine-tuning complete.")

Configuring training...
Starting fine-tuning...


c:\anaconda3\envs\finance_env\Lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.465100,0.401964,0.810094
2,0.218700,0.438567,0.824636
3,0.123500,0.519318,0.807528


c:\anaconda3\envs\finance_env\Lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\anaconda3\envs\finance_env\Lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Fine-tuning complete.


In [7]:
# --- 5. Evaluate the Final Model ---
print("\nEvaluating the fine-tuned model...")
eval_results = trainer.evaluate()
print(f"\nFinal evaluation accuracy: {eval_results['eval_accuracy'] * 100:.2f}%")


Evaluating the fine-tuned model...


c:\anaconda3\envs\finance_env\Lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



Final evaluation accuracy: 81.01%


In [ ]:
# --- 6. Save the Final Model ---
final_model_path = "./dev/models/finbert_finetuned_v1"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"Model saved to {final_model_path}")

Model saved to ./fine_tuned_categorizer
